# Ron Low

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("yahoo-finance.parquet")

In [3]:
yfinance_metrics = [
    "BTC-CAD_Low"
]

df = df[yfinance_metrics]

In [4]:
# Assuming df is your DataFrame after resampling and you've already dropped NA values
df.dropna(inplace=True)

# Add an assertion to ensure there are no NA values in the DataFrame
assert df.isnull().sum().sum() == 0, "DataFrame contains NA values"

df

,BTC-CAD_Low
Date,
2014-09-17,497.302246
2014-09-18,452.183655
2014-09-19,419.985870
2014-09-20,427.350739
2014-09-21,430.965698
...,...
2024-03-04,83437.976562
2024-03-05,84514.890625
2024-03-06,80644.625000


In [5]:
import pandas as pd
import numpy as np
import torch

from gluonts.dataset.common import ListDataset
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from gluonts.transform.feature import MissingValueImputation

torch.set_float32_matmul_precision('high')
                                   
# Assuming df is your DataFrame with the data
target_column = 'BTC-CAD_Low'  # Replace with your target column name

# Ensure the DataFrame's index is a datetime index and set the frequency explicitly if needed
df.index = pd.to_datetime(df.index)
freq = "D"  # Set the frequency of your data, e.g., 'D' for daily. Adjust as needed.
df = df.asfreq(freq)

# Define the prediction length
prediction_length = 1  # Set your prediction length

# Make sure the lengths match when creating ListDataset
training_data = ListDataset([
    {
        "start": df.index[0],
        "target": df[target_column][:-100]    
    }
], freq=freq)

test_data = ListDataset([
    {
        "start": df.index[-100],
        "target": df[target_column][-100:].values
    }
], freq=freq)

# Initialize the Temporal Fusion Transformer Estimator
estimator = TemporalFusionTransformerEstimator(
    freq=freq,
    prediction_length=prediction_length,
    context_length=60,  # Optional: adjust based on your needs
    num_heads=8,
    hidden_dim=256,
    variable_dim=256,
    quantiles=[0.1, 0.5, 0.9],  # Specifying the quantiles for forecasting
    lr=0.001,
    weight_decay=1e-08,
    dropout_rate=0.1,
    patience=10,
    batch_size=128,
    num_batches_per_epoch=100,
    trainer_kwargs={'max_epochs': 1000},  # Adjust 'gpus' based on your setup
)

# Train the model
predictor = estimator.train(training_data)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\falty\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
C:\Users\fa

Epoch 0: |                                               | 100/? [00:10<00:00,  9.11it/s, v_num=23, train_loss=1.68e+3]

Epoch 0, global step 100: 'train_loss' reached 1680.34814 (best 1680.34814), saving model to 'C:\\Users\\falty\\Desktop\\GZS\\trading\\options\\tft\\ron\\lightning_logs\\version_23\\checkpoints\\epoch=0-step=100.ckpt' as top 1


Epoch 1: |                                               | 100/? [00:10<00:00,  9.14it/s, v_num=23, train_loss=1.02e+3]

Epoch 1, global step 200: 'train_loss' reached 1020.32275 (best 1020.32275), saving model to 'C:\\Users\\falty\\Desktop\\GZS\\trading\\options\\tft\\ron\\lightning_logs\\version_23\\checkpoints\\epoch=1-step=200.ckpt' as top 1


...

Epoch 321, global step 32200: 'train_loss' reached 10.90798 (best 10.90798), saving model to 'C:\\Users\\falty\\Desktop\\GZS\\trading\\options\\tft\\ron\\lightning_logs\\version_23\\checkpoints\\epoch=321-step=32200.ckpt' as top 1


Epoch 322: |                                               | 100/? [00:11<00:00,  8.77it/s, v_num=23, train_loss=10.90]

Epoch 322, global step 32300: 'train_loss' was not in top 1


Epoch 323: |                                                | 14/? [00:01<00:00,  8.59it/s, v_num=23, train_loss=10.90]

C:\Users\falty\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [6]:
# Collect actual and predicted values for evaluation, including percentiles
actuals = df[target_column][-prediction_length:].values

mean_predictions = []
p10_predictions = []
p50_predictions = []
p90_predictions = []

for forecast in predictor.predict(test_data):
    p10_predictions.append(forecast.quantile(0.1))
    p50_predictions.append(forecast.quantile(0.5))  # Median
    p90_predictions.append(forecast.quantile(0.9))

# Convert lists to numpy arrays for slicing
p10_predictions = np.array(p10_predictions).flatten()[:prediction_length]
p50_predictions = np.array(p50_predictions).flatten()[:prediction_length]
p90_predictions = np.array(p90_predictions).flatten()[:prediction_length]

In [7]:
last_row = df.iloc[-prediction_length:, ]
last_row

,BTC-CAD_Low
Date,
2024-03-08,88748.6875


In [8]:
print(p10_predictions, p50_predictions, p90_predictions)

[86381.16] [86399.586] [86425.65]


In [9]:
# Function to calculate sMAPE
def calculate_smape(forecasts, actuals):
    return 100 * np.mean(2 * np.abs(forecasts - actuals) / (np.abs(actuals) + np.abs(forecasts)))

# Calculate standard evaluation metrics for mean predictions
mae = np.mean(np.abs(p50_predictions - actuals))
mape = np.mean(np.abs((p50_predictions - actuals) / actuals)) * 100
smape = calculate_smape(p50_predictions, actuals)

# Calculate the percentage of actuals within the 10th to 90th percentile range
within_range = np.sum((actuals >= p10_predictions) & (actuals <= p90_predictions)) / len(actuals) * 100

print("Evaluation Metrics for Mean Predictions:")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"sMAPE: {smape:.2f}%")
print(f"Percentage of Actuals within P10-P90 Interval: {within_range:.2f}%")


Evaluation Metrics for Mean Predictions:
MAE: 2349.10
MAPE: 2.65%
sMAPE: 2.68%
Percentage of Actuals within P10-P90 Interval: 0.00%


In [10]:
import os
import torch

# Define a path to save the model
model_save_path = 'ron_low.pth'

# Assuming 'predictor' is the trained model from DeepAREstimator
torch.save(predictor, model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to ron_low.pth


In [15]:
import pandas as pd
import numpy as np
from gluonts.dataset.common import ListDataset
# [other imports]

# [Assuming your DataFrame 'df' and the target_column are already defined]
# [Also assuming the estimator and model training are already done and you have 'predictor']

prediction_length = 1  # Forecasting 30 days into the future
context_length = 60  # The length of the history to consider for the prediction

# Rolling window prediction function
def perform_rolling_prediction(df, predictor, target_column, context_length, prediction_length):
    rolling_predictions = []
    prediction_dates = []
    temp_df = df.copy()  # Create a copy of the dataframe to modify

    # Create dates for predictions
    start_prediction_date = temp_df.index.max() + pd.Timedelta(1, unit='D')
    prediction_dates = pd.date_range(start=start_prediction_date, periods=30, freq='D')
    
    for i, prediction_date in enumerate(prediction_dates):
        end_idx = len(temp_df) - prediction_length + i
        test_data = ListDataset([
            {
                "start": temp_df.index[0],
                "target": temp_df[target_column][:end_idx].values
            }
        ], freq='D')

        forecast = next(predictor.predict(test_data))
        predicted_value = forecast.quantile(0.5)[-1]
        rolling_predictions.append(predicted_value)
        
        # Append the predicted value for future rolling windows
        new_row = pd.DataFrame({target_column: [predicted_value]}, index=[prediction_date])
        temp_df = pd.concat([temp_df, new_row])
    
    return prediction_dates, rolling_predictions

# Perform rolling predictions for the next 30 days
prediction_dates, rolling_predictions = perform_rolling_prediction(df, predictor, target_column, context_length, prediction_length)

# Print the rolling predictions
print("Rolling Predictions for the next 30 days:")
for date, prediction in zip(prediction_dates, rolling_predictions):
    print(f"{date.strftime('%Y-%m-%d')}: {prediction}")


Rolling Predictions for the next 30 days:
2024-03-09: 85854.7734375
2024-03-10: 85301.828125
2024-03-11: 84458.40625
2024-03-12: 84172.5625
2024-03-13: 85528.8125
2024-03-14: 86672.859375
2024-03-15: 87755.875
2024-03-16: 87347.6875
2024-03-17: 87482.8671875
2024-03-18: 87768.96875
2024-03-19: 88669.109375
2024-03-20: 85978.484375
2024-03-21: 81988.546875
2024-03-22: 80487.65625
2024-03-23: 81639.6015625
2024-03-24: 81308.0625
2024-03-25: 82444.7578125
2024-03-26: 80280.3984375
2024-03-27: 77718.5234375
2024-03-28: 78971.15625
2024-03-29: 82943.4921875
2024-03-30: 84596.1875
2024-03-31: 84849.3046875
2024-04-01: 84809.328125
2024-04-02: 82978.6796875
2024-04-03: 85267.0625
2024-04-04: 87138.4296875
2024-04-05: 88284.875
2024-04-06: 90338.71875
2024-04-07: 89542.5390625
